In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score

In [18]:
curr_folder = 'real1'
csv_path = f'/Users/ngocanh/Documents/Master/in progress/CS4220/scripts/{curr_folder}_final.csv'
NUM_TOP = 2
algo_dict = {
    'mutect2': 'FILTER_Mutect2',
    'varscan': 'FILTER_Varscan',
    'vardict': 'FILTER_Vardict',
    'freebayes': 'FILTER_Freebayes'
}

In [19]:
df = pd.read_csv(csv_path, delimiter='\t')


/var/folders/_0/ztgrcgnj1bd__vhjt7plwd2w0000gn/T/ipykernel_30267/3912418589.py:1: DtypeWarning: Columns (1,52) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path, delimiter='\t')


In [37]:
def get_f1(df, pred_col, true_col):
    y_pred = df[pred_col].to_numpy()
    y_true = df[true_col].to_numpy()
    return f1_score(y_true, y_pred)
    
def naive_algo_1(df):
    df['count_num_true'] = df[algo_dict.values()].sum(axis=1)
    df['pred_by_at_least_2'] = df.apply(lambda row: True if row['count_num_true'] > 1 else False, axis=1)
    print("\nF1 output from at least 2 algorithms:" )
    print(get_f1(df, 'pred_by_at_least_2', 'is_snv'))

def naive_algo_2(df):
    f1_score_dict = {}
    for algo, col in algo_dict.items():
        f1_score_dict[col] = get_f1(df, col, 'is_snv')

    print("F1 output from individual algos:")
    print(f1_score_dict)
    top_cols = dict(sorted(f1_score_dict.items(), key = lambda x: x[1], reverse = True)[:NUM_TOP])
    df['pred_by_top_2'] = df.apply(lambda row: all(row[col] for col in top_cols), axis=1)
    print(f"\nF1 output from top 2 algorithms: {top_cols.keys()}" )
    print(get_f1(df, 'pred_by_top_2', 'is_snv'))


In [38]:
naive_algo_2(df)

F1 output from individual algos:
{'FILTER_Mutect2': 0.3572997510616488, 'FILTER_Varscan': 0.013910441143870538, 'FILTER_Vardict': 0.26708727655099895, 'FILTER_Freebayes': 0.29435327561700475}

F1 output from top 2 algorithms: dict_keys(['FILTER_Mutect2', 'FILTER_Freebayes'])
0.8201117318435754


In [39]:
naive_algo_1(df)


F1 output from at least 2 algorithms:
0.5571658615136876
